In [1]:
import numpy as np
import numpy.typing as npt
from scipy.spatial.transform import Rotation
from typing import Dict, List, Tuple, Optional

In [2]:
def get_rotation_matrix(forward: npt.NDArray,
                        up: npt.NDArray) -> npt.NDArray:
    z = forward / np.sqrt(np.dot(forward, forward))
    y = up / np.sqrt(np.dot(up, up))
    x = np.cross(z, y)
    return np.column_stack((x, y, -z))

def rotate(rotation_matrix: npt.NDArray,
           vector: npt.NDArray) -> npt.NDArray:
    rot_vector = np.dot(rotation_matrix, vector)
    return rot_vector

In [3]:
UP = np.asarray([0, 1, 0])
DOWN = np.asarray([0, -1, 0])
RIGHT = np.asarray([1, 0, 0])
LEFT = np.asarray([-1, 0, 0])
FORWARD = np.asarray([0, 0, -1])
BACKWARD = np.asarray([0, 0, 1])

In [4]:
class MountPoint:
    def __init__(self,
                 face: npt.NDArray,
                 start: npt.NDArray,
                 end: npt.NDArray,
                 exclusion_mask: int,
                 properties_mask: int) -> None:
        self.face = face
        self.start = np.rint(start)
        self.end = np.rint(end)
        self.exclusion_mask = exclusion_mask
        self.properties_mask = properties_mask
        

class Block:
    def __init__(self,
                 block_type: str,
                 orientation_forward: npt.NDArray,
                 orientation_up: npt.NDArray,
                 position: npt.NDArray,
                 mountpoints: List[MountPoint]):
        self.block_type = block_type
        self.orientation_forward = orientation_forward
        self.orientation_up = orientation_up
        self.position = position
        self.mountpoints = mountpoints
        self.scaled_size = 1
        
    def __repr__(self) -> str:
        return f'{self.block_type} at {self.position}; OF {self.orientation_forward}; OU {self.orientation_up}'

In [5]:
def _get_mountpoint_limits(mountpoints: List[MountPoint],
                           block: Block,
                           rot_direction: npt.NDArray,
                           realign: bool = False) -> Tuple[List[npt.NDArray], List[npt.NDArray]]:
    starts, ends = [], []
    for mp in mountpoints:
        # fix indices of face according to block origin
        realigned_start = np.asarray([mp.start[0] if rot_direction[0] == -1 or rot_direction[0] == 0 else mp.start[0] - block.scaled_size[0],
                                      mp.start[1] if rot_direction[1] == -1 or rot_direction[1] == 0 else mp.start[1] - block.scaled_size[1],
                                      mp.start[2] if rot_direction[2] == -1 or rot_direction[2] == 0 else mp.start[2] - block.scaled_size[2]])
        realigned_end = np.asarray([mp.end[0] if rot_direction[0] == -1 or rot_direction[0] == 0 else mp.end[0] - block.scaled_size[0],
                                    mp.end[1] if rot_direction[1] == -1 or rot_direction[1] == 0 else mp.end[1] - block.scaled_size[1],
                                    mp.end[2] if rot_direction[2] == -1 or rot_direction[2] == 0 else mp.end[2] - block.scaled_size[2]])

        if realign:
            # rotate back to have everything on the grid frame
            inv_rot_matrix = Rotation.from_matrix(get_rotation_matrix(forward=block.orientation_forward,
                                                                      up=block.orientation_up)).inv().as_matrix()
            
            realigned_start = rotate(rotation_matrix=inv_rot_matrix,
                                     vector=realigned_start)
            realigned_end = rotate(rotation_matrix=inv_rot_matrix,
                                   vector=realigned_end)
            
        starts.append(realigned_start)
        ends.append(realigned_end)
        
    ordered_ends = [np.asarray([max(start[0], end[0]), max(start[1], end[1]), max(start[2], end[2])]) for (start, end) in zip(starts, ends)]
    ordered_starts = [np.asarray([min(start[0], end[0]), min(start[1], end[1]), min(start[2], end[2])]) for (start, end) in zip(starts, ends)]
    
    return ordered_starts, ordered_ends

In [6]:
def _try_and_get_block(idx: Tuple[int, int, int],
                       offset: npt.NDArray,
                       hull: npt.NDArray) -> Optional[Block]:
    new_idx = np.rint(idx + offset).astype(np.int32)
    return hull[new_idx[0], new_idx[1], new_idx[2]]

In [7]:
def _check_valid_placement(idx: Tuple[int, int, int],
                           block: Block,
                           direction: npt.NDArray,
                           hull: np.typing.NDArray) -> Tuple[bool]:
    rot_direction = rotate(get_rotation_matrix(forward=block.orientation_forward,
                                                up=block.orientation_up),
                            vector=direction)
    
    print(f'Block #1 - rotated direction (normal face): {rot_direction}')
    
    mp1 = [mp for mp in block.mountpoints if np.array_equal(mp.face, rot_direction)]
    starts1, ends1 = _get_mountpoint_limits(mountpoints=mp1,
                                                     block=block,
                                                     rot_direction=rot_direction,
                                                     realign=True)
    
    print(f'Block #1 - start and end mountpoints vectors: {starts1} : {ends1}')
    
    other_block = _try_and_get_block(idx=idx,
                                     offset=direction,
                                     hull=hull)
    if other_block is None:
        # facing air block, can always be placed
        return True
    else:
        # facing block but has no mountpoints in given direction
        if mp1 == []:
            return False

    opposite_direction = direction * (-1)
    rot_other = rotate(get_rotation_matrix(forward=other_block.orientation_forward,
                                            up=other_block.orientation_up),
                        vector=opposite_direction)
    
    print(f'Block #2 - rotated direction (normal face): {rot_other}')
    
    mp2 = [mp for mp in other_block.mountpoints if np.array_equal(mp.face, rot_other)]
    if mp2 == []:
        # other block has no mountpoints in given direction
        return False
    starts2, ends2 = _get_mountpoint_limits(mountpoints=mp2,
                                                     block=other_block,
                                                     rot_direction=rot_other,
                                                     realign=True)
    
    print(f'Block #2 - start and end mountpoints vectors: {starts2} : {ends2}')
    

    # assume block will always be "smaller" than other_block
    # then all mp1 should be contained in some mp2
    all_valid = []
    for eo1, so1 in zip(ends1, starts1):
        mp_valid = True
        for eo2, so2 in zip(ends2, starts2):
            # NOTE: This check does not take into account exclusions and properties masks (yet)
            mp_valid &= so1.x >= so2.x and eo1.x <= eo2.x and so1.y >= so2.y and eo1.y <= eo2.y and so1.z >= so2.z and eo1.z <= eo2.z
        all_valid.append(mp_valid)

    return all(all_valid)

In [8]:

hull = np.zeros(shape=(3,3,3),
                dtype=Block)

hull[1,1,1] = Block(
    block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    orientation_forward=np.asarray([0, 0, -1]),
    orientation_up=np.asarray([0, -1, 0]),
    mountpoints=[
        MountPoint(face=np.asarray([1 , 0 , 0]),
                   start=np.asarray([0.9996 , 0.001 , 0.999]),
                   end=np.asarray([1.0004 , 0.999 , 0.0009999871]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([-1 , 0 , 0]),
                   start=np.asarray([0.0004 , 0.001 , 0.001]),
                   end=np.asarray([-0.0004 , 0.999 , 0.999]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([0 , 1 , 0]),
                   start=np.asarray([0.001 , 0.9996 , 0.999]),
                   end=np.asarray([0.999 , 1.0004 , 0.0009999871]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([0 , -1 , 0]),
                   start=np.asarray([0.001 , 0.0004 , 0.001]),
                   end=np.asarray([0.999 , -0.0004 , 0.999]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([0 , 0 , -1]),
                   start=np.asarray([0.999 , 0.001 , 0.0004]),
                   end=np.asarray([0.0009999871 , 0.999 , -0.0004]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([0 , 0 , 1]),
                   start=np.asarray([0.001 , 0.001 , 0.9996]),
                   end=np.asarray([0.999 , 0.999 , 1.0004]),
                   exclusion_mask=0,
                   properties_mask=0),
    ],
    position=np.asarray([1,1,1])
)

hull[1,1,0] = Block(
    block_type='MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    orientation_forward=np.asarray([0, -1, 0]),
    orientation_up=np.asarray([0, 0, 1]),
    mountpoints=[
        MountPoint(face=np.asarray([0 , 0 , -1]),
                   start=np.asarray([0.999 , 0.001 , 0.0004]),
                   end=np.asarray([0.0009999871 , 0.999 , -0.0004]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([0 , -1 , 0]),
                   start=np.asarray([0.001 , 0.0004 , 0.001]),
                   end=np.asarray([0.999 , -0.0004 , 0.999]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([-1 , 0 , 0]),
                   start=np.asarray([0.0004 , 0.001 , 0.001]),
                   end=np.asarray([-0.0004 , 0.999 , 0.499]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([-1 , 0 , 0]),
                   start=np.asarray([0.0004 , 0.001 , 0.501]),
                   end=np.asarray([-0.0004 , 0.499 , 0.999]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([1 , 0 , 0]),
                   start=np.asarray([0.9996 , 0.001 , 0.999]),
                   end=np.asarray([1.0004 , 0.499 , 0.501]),
                   exclusion_mask=0,
                   properties_mask=0),
        MountPoint(face=np.asarray([1 , 0 , 0]),
                   start=np.asarray([0.9996 , 0.001 , 0.499]),
                   end=np.asarray([1.0004 , 0.999 , 0.0009999871]),
                   exclusion_mask=0,
                   properties_mask=0),
    ],
    position=np.asarray([1,1,0])
)

checking_direction = np.asarray([0, 0, -1])
idx = (1,1,1)

print(f'Checking {hull[idx]} in direction {checking_direction}')

res = _check_valid_placement(idx=idx,
                             block=hull[1,1,1],
                             direction=checking_direction,
                             hull=hull)

print(f'Valid placement? {res} -- Expected: True')

Checking MyObjectBuilder_CubeBlock_LargeBlockArmorBlock at [1 1 1]; OF [ 0  0 -1]; OU [ 0 -1  0] in direction [ 0  0 -1]
Block #1 - rotated direction (normal face): [ 0.  0. -1.]
Block #1 - start and end mountpoints vectors: [array([-1., -1.,  0.])] : [array([0., 0., 0.])]
Block #2 - rotated direction (normal face): [0. 1. 0.]
Valid placement? False -- Expected: True
